### Data scraping from hh.ru and preparation

[EN] The main goal of this file is to understand how one can get data about vacancies from the biggest Russian job site - hh.ru.<br>
Here I'll analyze the search results about DS vacancies. Transformed data will be saved in .csv for further processing<br>
[RU] Основная задача файла - разобраться, каким образом получить данные о вакансиях с hh.ru.<br>
Для этого я проанализирую результат поисковой выдачи по направлению DS. Данные, которые я смогу получить, сохраню затем в .csv для дальнейшей обработки

In [1]:
#import requests
import urllib.request
#import datetime
#import re
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd

from time import sleep

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

[EN] Preparing browser under selenium control to collect data<br>
[RU] Подготовка браузера под контролем библиотеки selenium к сбору данных

In [2]:
chrome_mode = 'headed' #'headless' # for debug purposes we can change this value to any but 'headless' to run Chrome in standard mode
chrome_options = Options()
if chrome_mode == 'headless':
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--headless')
service = Service(executable_path="c:\\Applications\\WebDriver\\chromedriver-x32.exe")
browser = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
# urls to search vacancies by words "аналитик данных" / "data scien*" in Russia
#https://hh.ru/search/vacancy?text=data+scien*&search_field=name&search_field=description&area=1&salary=150000&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=100&no_magic=true&L_save_area=true
#https://hh.ru/search/vacancy?text=data+scien*&search_field=name&search_field=description&area=1&salary=150000&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=100&no_magic=true&L_save_area=true&page=1&hhtmFrom=vacancy_search_list
#https://hh.ru/search/vacancy?text=data+scien*&search_field=name&search_field=description&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=100&no_magic=true&L_save_area=true
search_url_template = "https://hh.ru/search/vacancy?text={}&search_field=name&search_field=description&{}salary={}&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page={}"
items_on_page = 100
salary_level = '' #'150000'
search_texts = [
    "data+scien*",
    "%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85",
]
areas = {
    'All': '',
    'Moscow': 'area=1&',
    'SPb': 'area=2&',
    'Ekaterinburg': 'area=3&',
    'Novosib': 'area=4&',
    'Austria': 'area=7&',
    'Erevan': 'area=13&',
    'NNovgorod': 'area=66&',
    'RostovND': 'area=76&',
    'Samara': 'area=78&',
    'Saratov': 'area=79&',
    'Kazan': 'area=88&',
    'Chelyabinsk': 'area=104&',
    '???': 'area=159&',
    'Almaty': 'area=160&',
    'Minsk': 'area=1002&',
    'Nur-Sultan': 'area=159&',
    'Tbilisi': 'area=2758&',
    'Tashkent': 'area=2759&',
}
url_tail = '&page={}&hhtmFrom=vacancy_search_list'

In [4]:
def combine_base_url(template=None, search_text='', area_keys=[], salary='', items_per_page=100):
    if template is None:
        return None
    areas_str = ''
    for key in area_keys:
        areas_str += areas[key]
    return template.format(search_text, areas_str, salary, items_per_page)

In [5]:
base_url = combine_base_url(template = search_url_template, search_text=search_texts[0])
base_url

'https://hh.ru/search/vacancy?text=data+scien*&search_field=name&search_field=description&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=100'

In [6]:
output_filename = 'vacancies_data_scientist'

In [7]:
browser.get(base_url)

In [8]:
soup = BeautifulSoup(browser.page_source, 'html.parser')
# Page source code showed us there is only 1 'template' tag on page.
# It contains a huge amount of data including vacancies list in dictionary-like format (possibly for JS parsing).
# So here I'll use json library to convert html text to dictionaries/lists
json_parsed = json.loads(soup.find_all('template')[0].text)
print('"Template" tag contains {} keys'.format(len(json_parsed)))

"Template" tag contains 496 keys


In [9]:
json_parsed

{'authPhone': None,
 'authNewEmployerAreaIdsToRedirect': [],
 'authNewEmployerCategories': [],
 'authNewEmployerFields': [],
 'authNewEmployerInitialValues': {},
 'authNewEmployerPhoneMask': None,
 'activeResumeAccessType': None,
 'accountTemporarilyLocked': {},
 'accountPhoneVerification': None,
 'applicantSignup': {'fields': [], 'hideLogin': False},
 'applicantVacancyResponseStatuses': {},
 'applicantResumes': [],
 'applicantResponseStreaks': {},
 'applicantPackageType': 'basic',
 'applicantServiceType': '',
 'applicantPaymentBackUrl': '',
 'applicantAnalyticsAction': '',
 'applicantPaymentTypes': [],
 'applicantAvailableResumeServices': [],
 'applicantPackageContent': [],
 'applicantAvailableQuantities': [],
 'applicantServicesPrices': {},
 'applicantPaymentSource': 'desktop',
 'applicantFindJobRecommendedQuantity': None,
 'applicantSuitableVacancyByResume': {},
 'account': {'firstName': None, 'middleName': None, 'lastName': None},
 'accountConnect': {},
 'accountConnectOAuth': {},


In [12]:
# Total number of search results
print(json_parsed['searchCounts'])
# or
print(json_parsed['vacancySearchResult']['totalResults'])
# ?

{'isLoad': False, 'value': 1030}
1030


In [14]:
json_parsed['searchClustersBasic'].keys()

dict_keys(['label', 'industry', 'experience', 'schedule', 'professionalArea', 'professional_role', 'area', 'employment', 'compensation', 'part_time', 'search_field', 'excluded_text'])

[EN] In case total search results number exceeds 2000 (seems to be hardcoded limitation) it is possible to use 'searchClustersBasic'->'area' to implement partial searches  
[RU] Если итоговое количество записей превысит 2000 (а это, похоже, "жестко" прописанное ограничение), можно пропарсить ключ 'searchClustersBasic'->'area', чтобы организовать частичные выборки и затем объединить результаты

In [16]:
json_parsed['searchClustersBasic']['area']['groups'].keys()

dict_keys(['1', '2', '7', '9', '13', '16', '21', '27', '28', '37', '40', '48', '74', '85', '94', '97', '113', '146', '150', '152', '153', '159', '160', '172', '177', '180', '181', '188', '194', '199', '200', '205', '208', '236', '1001', '1002', '1146', '1202', '1217', '1255', '1261', '1317', '1384', '1438', '1511', '1530', '1586', '1596', '1624', '1646', '1652', '1661', '1679', '1704', '1716', '1844', '1880', '1898', '1913', '1948', '2237', '2492', '2758', '2759', '2760', '2814', '5046'])

In [18]:
json_parsed['searchClustersBasic']['area']['groups']['1']

{'count': 527, 'seoDomain': 'hh.ru', 'order': 2, 'title': 'Москва', 'id': '1'}

'count' - vacancies found in area, 'id' - area id (we can add records to _areas_ dictionary using 'id' and 'title' keys to make it possible to search in more regions)

[EN] We've got a huge amount of 'empty' data structures after transforming. There are empty dictionaries and dictionaries which contain 'empty' data structures. So I'll clean these artifacts out to make the search of data more efficient.<br>
[RU] В полученном ответе много "пустых" данных - пустые словари, а также словари, содержащие "пустые" структуры. Очистим результат от этих артефактов, чтобы было легче искать значимую информацию

In [19]:
# Function to check if dictionary is 'empty' / Функция, проверяющая словарь на "пустоту"
def is_dict_empty(input_dict):
    result = True
    for key in input_dict.keys():
        if (type(input_dict[key]) is type(dict())) and (len(input_dict[key]) > 0):
            # Рекуррентная проверка словарей
            result = result and is_dict_empty(input_dict[key])
        else:
            # "Пустыми" считать структуры, длина которых равна 0, имеющие значение None или являющиеся пустым словарем или списком
            checks_empty = (input_dict[key] is None) or (str(input_dict[key]) in ['{}', '[]']) or (len(str(input_dict[key])) == 0)
            result = result and checks_empty
        if not result:
            break
    return result

In [20]:
clean_dict = {}
for key in json_parsed.keys():
    if (type(json_parsed[key]) is type(dict())):
        if not is_dict_empty(json_parsed[key]):
            clean_dict[key] = json_parsed[key]
    else:
        checks_empty = (json_parsed[key] is None) or (str(json_parsed[key]) in ['{}', '[]']) or (len(str(json_parsed[key])) == 0)
        if not checks_empty:
            clean_dict[key] = json_parsed[key]
print('{} non-empty keys in result'.format(len(clean_dict)))
print('====================================================')
for key in clean_dict.keys():
    print('{} ====> {}'.format(key, clean_dict[key]))

181 non-empty keys in result
applicantSignup ====> {'fields': [], 'hideLogin': False}
applicantPackageType ====> basic
applicantPaymentSource ====> desktop
accountHistoryReplenishments ====> {'bills': [], 'documentLinksVisibility': False, 'currency': 'RUR'}
accountDelete ====> {'applicantName': '', 'resumesList': {'resumes': {'published': [], 'unpublished': []}, 'count': 0}}
adsSearchParams ====> {'puid11': 'searchVacancy', 'puid23': '', 'puid14': 'data scien*', 'puid29': '', 'puid30': '', 'puid12': '', 'puid13': ''}
advancedSearch ====> {'showSearchConditions': False, 'hideSuggest': False, 'vacancy': None, 'resume': None, 'experience': [], 'keySkills': [], 'university': [], 'citizenship': [], 'work_ticket': [], 'employment': [], 'schedule': [], 'driver_license_types': [], 'job_search_status': [], 'language': [], 'exclusion': []}
appleBusinessChat ====> {'isEnabled': False, 'href': ''}
abortPageContent ====> False
addressesSuggestRemoteMode ====> False
analyticsParams ====> {'hhtmSourc

[EN] Keys analysis show search results are under 'vacancySearchResult'->'vacancies' keys  
Another useful keys are:
'searchClusters' contains grouping characteristics
- 'industry'
- 'groups'  

[RU] Визуальный анализ ключей показывает, что результаты поиска хранятся в ключе 'vacancySearchResult'->'vacancies'  
Другие полезные ключи:
'searchClusters' - содержит группировочные характеристики
- 'industry' - список отраслей/направлений, в которых были найдены вакансии (возможно, с указанием количества?)
- 'groups' - список населенных пунктов

In [21]:
vacancies_info = clean_dict['vacancySearchResult']['vacancies']
print('Vacancies data type: {}'.format(type(vacancies_info)))
print('Num of vacancies: {}'.format(len(vacancies_info)))

Vacancies data type: <class 'list'>
Num of vacancies: 100


[EN] We have 100 records containing vacancies info from the 1st search page. Now we have to find total number of pages to get all of them. So we go to 'paging' key  
[RU] У нас имеются данные о вакансиях с 1 страницы, содержащие 100 записей, как мы и просили в запросе.  
Теперь нужно найти информацию о том, сколько страниц всего сформировано, чтобы организовать получение информации с остальных страниц.  
Для этого служит ключ 'paging'

In [22]:
clean_dict['vacancySearchResult']['paging']

{'previous': {'page': -1, 'disabled': True},
 'pages': [{'text': '1', 'page': 0, 'selected': True, 'inShortRange': True},
  {'text': '2', 'page': 1, 'selected': False, 'inShortRange': True},
  {'text': '3', 'page': 2, 'selected': False, 'inShortRange': True},
  {'text': '4', 'page': 3, 'selected': False, 'inShortRange': False},
  {'text': '5', 'page': 4, 'selected': False, 'inShortRange': False},
  {'text': '...', 'page': 5, 'selected': False, 'inShortRange': False}],
 'lastPage': {'page': 10, 'selected': False},
 'next': {'page': 1, 'disabled': False},
 'os': 'Win'}

[EN] Here is pagination data. Last page number contains in 'lastPage'->'page' key. So we have N+1 pages as they are 0-indexed.  
[RU] Здесь приводятся данные разметки по страницам. Номер последней страницы содержится в ключе 'lastPage'->'page'. Таким образом, всего в результатах поиска N+1 страниц, поскольку страница, которую мы получили, и анализируем сейчас, имеет индекс 0

In [23]:
last_page = clean_dict['vacancySearchResult']['paging']['lastPage']['page']
for page in range(1, last_page+1):
    browser.get(base_url+url_tail.format(page))
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    json_parsed = json.loads(soup.find_all('template')[0].text)
    vacancies_info += json_parsed['vacancySearchResult']['vacancies']

print('Final vacancies info contains {} record(s)'.format(len(vacancies_info)))

Final vacancies info contains 1028 record(s)


[EN] We've got info from all pages and now can close browser

In [24]:
browser.quit()

[EN] Dumping data to have an opportunity to restore raw data later...

In [25]:
with open('datasets/'+output_filename+'.json', 'w') as f:
    json.dump(vacancies_info, f)

[EN] And now it's time to select fields to fill in DataFrame  
[RU] Сейчас нужно понять, какие поля из описаний вакансий нам понадобятся, чтобы использовать их в анализе

In [26]:
for key in vacancies_info[0].keys():
    print('{} =======> {}'.format(key, vacancies_info[0][key]))

@responseLetterRequired =======> False
@showContact =======> False
vacancyId =======> 67570281
name =======> Data engineer (RnD)
company =======> {'@showSimilarVacancies': True, '@trusted': True, '@category': 'COMPANY', '@countryId': 4, '@state': 'APPROVED', 'id': 672796, 'name': 'SOFTSWISS', 'visibleName': 'SOFTSWISS', 'logos': {'logo': [{'@type': 'searchResultsPage', '@url': '/employer-logo/3921983.png'}, {'@type': 'small', '@url': '/employer-logo/3921983.png'}, {'@type': 'vacancyPage', '@url': '/employer-logo/3921984.png'}, {'@type': 'medium', '@url': '/employer-logo/3921984.png'}, {'@type': 'ORIGINAL', '@url': '/employer-logo-original/870277.png'}, {'@type': 'employerPage', '@url': '/employer-logo/3921982.png'}]}, 'companySiteUrl': 'https://careers.softswiss.com/'}
compensation =======> {'noCompensation': {}}
publicationTime =======> {'@timestamp': 1657183767, '$': '2022-07-07T11:49:27.850+03:00'}
type =======> open
area =======> {'@id': 2758, 'name': 'Тбилиси', 'path': '.28.2758.'

Полезную информацию несут / Useful info keys:
- 'vacancyId' - уникальный идентификатор вакансии, по которому потом можно перейти на страницу с подробным описанием вакансии
- 'name' - наименование вакансии (предполагаемая должность)
- 'company'->'visibleName' + 'company'->'department'->'@name'
- 'area'->'@id', 'area'->'name' - код и название условной географической области поиска
- 'address'->'displayName', 'address'->'marker'->('@lat', '@lng') - показываемый адрес и координаты для карты
- 'compensation'->{'from', 'to', 'currencyCode', 'gross'=(True=до вычета налогов, False=на руки)} (или 'compensation'->'noCompensation', если данных нет) - размер з/п
- 'workSchedule' - график занятости (полный / сменный / гибкий / удаленный...)
- 'snippet' - отрывки из описания вакансии dict('req' - требования, 'resp' - функции/задачи, 'cond' - условия, 'skill' - ?требуемые навыки?, 'desc' - ?)
- 'publicationTime'
- 'lastChangeTime'

Lets define function to get necessary data from json:

In [27]:
df_column_names = [
    'vacancy_id',
    'vacancy_name',
    'company_name',
    'company_dept',
    'area',
    'address',
    'latitude',
    'longitude',
    'salary_from',
    'salary_to',
    'salary_currency',
    'salary_gross',
    'publication_time',
    'last_changed',
    'schedule',
    'req',
    'resp',
    'cond',
    'skills'
]

In [28]:
def get_record_data(rec):
    result = dict()
    result['vacancy_id'] = rec['vacancyId']
    result['vacancy_name'] = rec['name']
    result['company_name'] = rec['company']['visibleName']
    if rec['company'].get('department', np.NAN) is np.NAN:
        result['company_dept'] = np.NAN
    else:
        result['company_dept'] = rec['company']['department'].get('@name', np.NAN)
    result['area'] = rec['area']['@id']
    if rec.get('address', None) is None:
        result['address'] = np.NAN
    else:
        result['address'] = rec['address'].get('displayName', np.NAN)
        if rec['address'].get('marker', None) is None:
            result['latitude'] = np.NAN
            result['longitude'] = np.NAN
        else:
            result['latitude'] = rec['address']['marker'].get('@lat', np.NAN)
            result['longitude'] = rec['address']['marker'].get('@lng', np.NAN)
    if rec['compensation'].get('noCompensation', None) is None:
        result['salary_from'] = rec['compensation'].get('from', np.NAN)
        result['salary_to'] = rec['compensation'].get('to', np.NAN)
        result['salary_currency'] = rec['compensation'].get('currencyCode', np.NAN)
        result['salary_gross'] = rec['compensation'].get('gross', np.NAN)
    else:
        result['salary_from'] = np.NAN
        result['salary_to'] = np.NAN
        result['salary_currency'] = np.NAN
        result['salary_gross'] = np.NAN
    result['publication_time'] = rec['publicationTime']['@timestamp']
    result['last_changed'] = rec['lastChangeTime']['@timestamp']
    result['schedule'] = rec['workSchedule']
    result['req'] = rec['snippet'].get('req', np.NAN)
    result['resp'] = rec['snippet'].get('resp', np.NAN)
    result['cond'] = rec['snippet'].get('cond', np.NAN)
    result['skills'] = rec['snippet'].get('skills', np.NAN)

    return result

In [29]:
raw_parsed_data = {name: [] for name in df_column_names}
for rec in vacancies_info:
    parsed = get_record_data(rec)
    for key in df_column_names:
        raw_parsed_data[key].append(parsed.get(key, np.NAN))
print('Control of num of records created:', len(raw_parsed_data['vacancy_id']))
print('Vacancies ID sample: ', raw_parsed_data['vacancy_id'][:10])

Control of num of records created: 1028
Vacancies ID sample:  [67570281, 54418768, 54168093, 66986986, 66198333, 66933586, 67613430, 67609341, 67617590, 67114858]


Converting data structures created to pandas DataFrame. Then making readable data in 'publication_time' and 'last_changed' columns. And finally assigning 'vacancy_id' as primary index (it's unique for each record). It will be useful later if I'll decide to update data so I'll be able to filter out existing data or update existing records

In [30]:
df = pd.DataFrame(raw_parsed_data)
df['publication_time'] = df['publication_time'].apply(pd.to_datetime, unit='s')
df['last_changed'] = df['last_changed'].apply(pd.to_datetime, unit='s')
df.set_index('vacancy_id', inplace=True)
df.head()

,vacancy_name,company_name,company_dept,area,address,latitude,longitude,salary_from,salary_to,salary_currency,salary_gross,publication_time,last_changed,schedule,req,resp,cond,skills
vacancy_id,,,,,,,,,,,,,,,,,,
67570281,Data engineer (RnD),SOFTSWISS,NaN,2758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07 08:49:27,2022-07-07 14:56:20,FULL_DAY,"Experience in data environments, such as Data ...",...Big Data technology landscape to build busi...,None,NaN
54418768,Senior data scientist,Банк ДОМ.РФ,NaN,1,"Москва, улица Воздвиженка, 10",55.753301,37.606263,NaN,NaN,NaN,NaN,2022-07-05 06:04:39,2022-07-05 06:04:39,FULL_DAY,Опыт от 1 года в банковской сфере. Знание стат...,Построение моделей вероятности дефолта (PD) и ...,Конкурентный уровень заработной платы + премии...,NaN
54168093,Middle / Senior QA-инженер в команду Поиска,HeadHunter::QA,HeadHunter::QA,1,"Москва, улица Годовикова, 9с10",55.809343,37.628505,200000.0,NaN,RUR,True,2022-07-02 18:44:56,2022-07-02 18:44:56,FULL_DAY,Знание методов и методик тестирования. Опыт те...,"Обнаружение, документирование и отслеживание д...",Возможность выбора места работы: удаленно или ...,NaN
66986986,Data Scientist,ImagiON,NaN,1,"Москва, Центральный административный округ, Пр...",55.749451,37.542824,NaN,NaN,NaN,NaN,2022-07-08 08:46:42,2022-07-08 08:59:43,FULL_DAY,Опыт работы от 1 года. Опыт работы по анализу ...,None,"Комфортный офис в Москва Сити, м. Деловой цент...",NaN
66198333,Data Scientist,БиАйЭй-Технолоджиз,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08 07:46:57,2022-07-08 07:46:57,FULL_DAY,Знание SQL. Знание математической статистики. ...,"Прогнозирование спроса, эластичность спроса по...",Бесценный опыт работы над интересными проектам...,NaN


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 67570281 to 66733561
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   vacancy_name      1028 non-null   object        
 1   company_name      1028 non-null   object        
 2   company_dept      206 non-null    object        
 3   area              1028 non-null   int64         
 4   address           405 non-null    object        
 5   latitude          404 non-null    float64       
 6   longitude         404 non-null    float64       
 7   salary_from       195 non-null    float64       
 8   salary_to         118 non-null    float64       
 9   salary_currency   221 non-null    object        
 10  salary_gross      220 non-null    object        
 11  publication_time  1028 non-null   datetime64[ns]
 12  last_changed      1028 non-null   datetime64[ns]
 13  schedule          1028 non-null   object        
 14  req          

In [32]:
df.to_csv('datasets/'+output_filename+'.csv')